In [1]:
# IMPORT IMAGES - ASSUMES IMGMAP CONTAINS VISUAL SIMILARITY VARIABLE (SEP. NOTEBOOK)
import json

imgMap = {}
with open('./Documents/OmarGit/parsons-machine-learning/final/data-mapping/posterAttrPy.json') as json_file:
    imgMap = json.load(json_file)
print(len(imgMap))

5142


In [2]:
# RUN THIS TO DO ONE HOT ENCODING
# NEEDED AFTER FRESH SCRAPE RUN
    
def checkAndAdd(item, l):
    if item not in l:
        l.append(item)
        
distinctIcons = [] 
distinctColl = []
distinctArtists = []

for im in imgMap:
    im = imgMap.get(im)
    icon = im.get('Iconography')
    if icon:
        if isinstance(icon, str):
            checkAndAdd(icon, distinctIcons)
        elif isinstance(icon, list):
            for i in icon:
                checkAndAdd(i, distinctIcons)
 
    coll = im.get('Special Collection')
    if coll:
        if isinstance(coll, str):
            checkAndAdd(icon, distinctColl)
        elif isinstance(coll, list):
            for i in coll:
                checkAndAdd(i, distinctColl)
                
    artist = im.get('Artist')
    if artist:
        if isinstance(artist, str):
            checkAndAdd(artist, distinctArtists)
        elif isinstance(artist, list):
            for i in artist:
                checkAndAdd(i, distinctArtists)
    
iconMap = {}
for ic in distinctIcons:
    iconMap[ic] = 0
    
collectionMap = {}
for ic in distinctColl:
    collectionMap[ic] = 0
    
artistMap = {}
for ic in distinctArtists:
    artistMap[ic] = 0
    
    
# ENCODE ICONS, COLLECTIONS, ARTISTS TO COLUMNS

from copy import copy    

for i in imgMap:
    mapInstance = copy(iconMap)
    i = imgMap.get(i);
    if i.get('Iconography'):
        for icon in i.get('Iconography'):
            mapInstance[icon] = 1
        i['EncodedIcons'] = mapInstance
    else:
        i['EncodedIcons'] = copy(iconMap)
        
    collInstance = copy(collectionMap)
    if i.get('Special Collection'):
        for coll in i.get('Special Collection'):
            collInstance[coll] = 1
        i['EncodedCollection'] = collInstance
    else:
        i['EncodedCollection'] = copy(collectionMap)
        
    artistInstance = copy(artistMap)
    if i.get('Artist'):
        for artist in i.get('Artist'):
            artistInstance[artist] = 1
        i['EncodedArtist'] = artistInstance
    else:
        i['EncodedArtist'] = copy(artistMap)
                
                  
# ENCODE DATES TO INT

# for i in imgMap:
#     i = imgMap.get(i);
#     print(i)
#     i['EncodedDate'] = int(i.get['Date'])
#     i['EncodedDate'] = int
#     year = i.get('Year')[0]
#     if year:
#         num = [int(i) for i in year.split() if i.isdigit()][0]
#         i['EncodedDate'] = int(num)
        
       

In [3]:
imgMap.get('338673932409').get('EncodedArtist')

{'Research in Progress ': 1,
 'Ibrahim  Ghannam': 0,
 ' Izzat': 0,
 'Abboud ': 0,
 'Ismail Shammout (1930-2006)': 0,
 'Rami  Abbas': 0,
 'Motlaq ': 0,
 'Jamil Shammout': 0,
 'Ali Al Khatib': 0,
 'Kamal Boullata': 0,
 'Shafik Radwan': 0,
 'Samir Salameh (1944-2018)': 0,
 'Natheer Nabaa': 0,
 'P. P.': 0,
 'Ghassan Kanafani': 0,
 'Khalil Zgaib': 0,
 'Yusuf Al Nasser': 0,
 'Rafeik Sharaf': 0,
 'Jumana El Husseini': 0,
 'Al Muhandis Shukri ': 0,
 'Aref Rayess': 0,
 'Hani Jawharieh': 0,
 'Shami': 0,
 'Stelio Scamanga': 0,
 'Ahmad Shebrin': 0,
 ' Baayoun': 0,
 'Vladimir Tamari': 0,
 'Mustafa Al Hallaj (1938-2002)': 0,
 'Zaid  Wahba': 0,
 'P.E.': 0,
 'Samir Nasr': 0,
 'Qasem Abdelqader': 0,
 'Guy Le Querrec': 0,
 'Emad Abdel Wahhab': 0,
 'Mohammad  Ghani': 0,
 'Rocinante': 0,
 'Eid Muhammad': 0,
 'Bruno Barbey': 0,
 'Hamdi  Al Kassab': 0,
 'Theresa ': 0,
 'Marc Rudin': 0,
 'Naim Ismael': 0,
 'Elisabetta Carboni (Montaldo)': 0,
 'Abdel Kader Arnaout': 0,
 'Hassan Abu Aish ': 0,
 'Alberr  Shomal

In [4]:
# FEATURES ARRAY FOR METADATA, UNREDUCED

featuresToProcess = ['EncodedIcons']
featuresAll = {}

for feat in featuresToProcess:
    featuresLocal = []
    for i in imgMap:
        imgObj = imgMap.get(i)
        featuresList = list(imgObj.get(feat).values())
        featuresLocal.append(featuresList)
    featuresAll[feat] = featuresLocal
        

In [5]:
# create aggregated metadata array

import numpy as np
encodedFeatures = ['EncodedIcons']

featuresMetadata = []

for (index, val) in enumerate(imgMap):
    aggregatedFeatures = []
    for currFeat in encodedFeatures:
        featureValues = list(imgMap.get(val).get(currFeat).values()) 
        aggregatedFeatures = aggregatedFeatures + featureValues
    featuresMetadata.append(aggregatedFeatures)
        

In [6]:
# now get image array

# IMAGES FEATURES ARRAY

import numpy

import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.models import Model
model = keras.applications.VGG16(weights='imagenet', include_top=True)
feat_extractor = Model(inputs=model.input, outputs=model.get_layer('fc2').output)

# GET EACH IMAGE FROM DIRECTORY, AND CALCULATE FEATURES
from PIL import Image

featuresImg = []
images = [] #imagePath
numFailed = 0
successImgDir = {}

filePrefix = './Documents/OmarGit/parsons-machine-learning/final/poster-assets/'
fileSuffix = '.jpg'

print('start')
for i, imgInput in enumerate(imgMap):
 
    if i % 100 == 0:
        print(str(i))   
        
    try:      
        imgInput = imgMap.get(imgInput)
        image_path =  filePrefix + str(imgInput.get('Id')) + fileSuffix
        #loads image and resizes to res
        img = image.load_img(image_path, target_size=model.input_shape[1:3])

        #adds 3 channels for rgb to image
        x = image.img_to_array(img) 

        # wrap array in array reshape it so it's one image per array item
        x = np.expand_dims(x, axis=0) 

        # subtracts mean rgb channels of dataset (model trained diff)
        x = preprocess_input(x)

        # extracts 4096 features from image (total res is 50k so it is reduced)
        feat = feat_extractor.predict(x)[0] 
        featuresImg.append(feat)
  
        successImgDir[imgInput.get('Id')] = imgInput
        images.append(image_path)    
    except:
        print('failing', image_path)
        featuresImg.append(numpy.zeros(4096))
        images.append(image_path)    
        numFailed += 1

print('done')
print('failed', numFailed)

start
0
100
200
300
400
500
600
700
800
900
1000
1100
failing ./Documents/OmarGit/parsons-machine-learning/final/poster-assets/919283599879.jpg
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
failing ./Documents/OmarGit/parsons-machine-learning/final/poster-assets/190740779824.jpg
3700
3800
3900
failing ./Documents/OmarGit/parsons-machine-learning/final/poster-assets/164879825630.jpg
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
done
failed 3


In [7]:
# COMBINE METADATA AND VISUAL FEATURES
featuresCombined = []
featureHolder = []

#combine metadata first
for feature in featuresAll:
    feature = featuresAll.get(feature)
    for (i, item) in enumerate(feature):
        if (len(featureHolder) < i+1):
            featureHolder.append([])
        featureHolder[i] = featureHolder[i] + [x for x in item]
        
        
#combine with imgDAta  
imgList = list(featuresImg)
for (i, item) in enumerate(imgList):
    featureHolder[i] = featureHolder[i] + [x for x in item]

print(len(featureHolder))
#[item for sublist in regular_list for item in sublist]


5142


In [10]:
len(featureHolder[5])

4135

In [12]:
# REDUCE THOSE COMBINED ARRAYS lol

from sklearn.preprocessing import StandardScaler
import umap

ss = StandardScaler()
scaled = ss.fit_transform(featureHolder)
embedding = umap.UMAP().fit_transform(scaled)

#recale from 0 to 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(embedding)
embedding_scaled = scaler.transform(embedding)

In [ ]:
# UMAP > GRID

import rasterfairy
nx = 93
ny = 56

grid_assignment = rasterfairy.transformPointCloud2D(embedding, target=(nx, ny))

for (i, item) in enumerate(imgMap):
    grid_assign = list(grid_assignment[0][i])
    for (i, coord) in enumerate(grid_assign):
        grid_assign[i] = int(coord)
    imgMap.get(item)['GridPosX'] = grid_assign[0]
    imgMap.get(item)['GridPosY'] = grid_assign[1]

In [13]:
## NEW CLEAN PROCESSED OUTPUT ARRAY

imgMapOutput = {}

for (index, i) in enumerate(imgMap):
    i = imgMap.get(i)
    
    imgObj = {
        'Id': i.get('Id'),
        'Title': i.get('Title'),
        'Link': i.get('Link'),                
        'Thumb': i.get('Id'),
        'FullImage': i.get('FullImage'),    
        'Translation/Interpretation/Caption Text': i.get('Translation/Interpretation/Caption Text'),  
        'Date': i.get('Date'),       
    
        #array data now
        'Artist': i.get('Artist'),  
        'Source': i.get('Source'),    
        'Iconography': i.get('Iconography'), 
        'Special Collection': i.get('Special Collection'),  
        'Language': i.get('Language'),       
        
        #reduced data
        'GridPosX': i.get('GridPosX'),
        'GridPosY': i.get('GridPosY'),
    }
    imgMapOutput[i.get('Id')] = imgObj
    

with open('./Documents/OmarGit/parsons-machine-learning/final/data-mapping/posterAttrGridFlat.json', 'w') as outfile:
    json.dump(imgMapOutput, outfile)
print('done')

done
